In [33]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding, Conv1D
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer

import os
import zipfile

In [2]:
#NN custom functions
import Tools.NN as nn

In [3]:
df = pd.read_csv('./body')
df_target = pd.read_csv('./target')

In [4]:
original_df= df.merge(df_target, on = 'Unnamed: 0')

In [5]:
original_df.drop('Unnamed: 0',axis =1 , inplace = True)

In [6]:

# def clean_data(review):
#     review = re.sub('[^a-zA-Z]', ' ',review)
#     review = review.lower()
#     return review

In [7]:
original_df.head()

,review,target
0,Looking for some authentic Japanese food at re...,1
1,Pepe Rosso is where you go when you're in SOHO...,1
2,I had waited to return a couple other times to...,1
3,This place is always busy - partly because it'...,1
4,Love this place! I am not a regular yelper I d...,0


In [8]:
original_df['review'] = original_df['review'].apply(nn.clean_data)

In [9]:
original_df.target.value_counts()

1    256887
0     29107
Name: target, dtype: int64

In [10]:
#download nltk stopwords in necessary
#nltk.download('stopwords')

In [11]:
# sw = stopwords.words('english')
# sw += ['food','place','good',"great"]

In [12]:
# def remove_stop_words(review):
#     review_minus_sw = []
#     stop_words = sw
#     review = review.split()
#     review = [review_minus_sw.append(word) for word in review if word not in stop_words]
#     review = ' '.join(review_minus_sw)
#     return review

In [13]:
original_df['review'] = original_df['review'].apply(nn.remove_stop_words)

In [14]:
corpus = list(original_df['review'])

In [15]:
cv = CountVectorizer(max_features = 1000)
 # Stop words are common words in English that don't tell us anything about the polarity of a review.
    # Such words include the, that, and a
# Converts a collection of text documents to a matrix of token counts
# max_features = maximum number of words we'd like to have in our bag of words model
X = cv.fit_transform(corpus).toarray()
y = original_df['target'].values

In [16]:
#features of the count vectorizers
#cv.get_feature_names()

['able',
 'absolutely',
 'accommodating',
 'across',
 'actually',
 'add',
 'added',
 'addition',
 'affordable',
 'afternoon',
 'ago',
 'al',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'amazing',
 'ambiance',
 'ambience',
 'american',
 'amount',
 'another',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'apparently',
 'appetizer',
 'appetizers',
 'apple',
 'area',
 'around',
 'arrived',
 'artichoke',
 'asian',
 'ask',
 'asked',
 'ate',
 'atmosphere',
 'attention',
 'attentive',
 'authentic',
 'available',
 'average',
 'avocado',
 'away',
 'awesome',
 'baby',
 'back',
 'bacon',
 'bad',
 'baked',
 'balls',
 'banana',
 'banh',
 'bar',
 'barely',
 'bartender',
 'bartenders',
 'based',
 'basically',
 'basil',
 'bathroom',
 'bbq',
 'bean',
 'beans',
 'beat',
 'beautiful',
 'beef',
 'beer',
 'beers',
 'behind',
 'believe',
 'belly',
 'benedict',
 'best',
 'better',
 'beyond',
 'bf',
 'big',
 'bill',
 'birthday',
 'biscuit',
 'bit',
 'bite',
 'bites',
 'black

In [17]:
tf_transformer = TfidfTransformer()
X = tf_transformer.fit_transform(X).toarray()

In [18]:

tfidfVectorizer = TfidfVectorizer(max_features =1000)
X = tfidfVectorizer.fit_transform(corpus).toarray()

In [19]:
# from sklearn.model_selection import train_test_split
# X_train_s, X_test_s , y_train_s, y_test_s = train_test_split(X, y, random_state=42, stratify = y)

In [20]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.naive_bayes import GaussianNB
# # Naive Bayes is a statistical classification technique based on Bayes Theorem
# # common classifier used in sentiment analysis is the Naive Bayes Classifier.
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.ensemble import HistGradientBoostingClassifier # this is experimental
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# classifiers = [GradientBoostingClassifier(),GaussianNB(),HistGradientBoostingClassifier(),
#                RandomForestClassifier(),LogisticRegression()]

In [21]:
# from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
# for classifier in classifiers:
#     classifier.fit(X_train_s,y_train_s)
#     print(f'The {classifier}  Accuracy  is {accuracy_score(y_test_s,classifier.predict(X_test_s)) }' )

In [22]:
docs = original_df['review']
labels = original_df['target']
X_train, X_test , y_train, y_test = train_test_split(docs, labels , random_state = 42, stratify = labels)

In [23]:
vocab_size = 5000

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]

In [24]:

max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [25]:

model1 = Sequential([
    Embedding(vocab_size, 8, input_length=max_length),
   Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
  Dense(10, activation='relu'),
  Dense(1, activation='sigmoid')
])

In [26]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [27]:
history1 = model1.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
6703/6703 [==============================] - 27s 4ms/step - loss: 0.3258 - acc: 0.8954 - val_loss: 0.3038 - val_acc: 0.8982
Epoch 2/20
6703/6703 [==============================] - 28s 4ms/step - loss: 0.2979 - acc: 0.8971 - val_loss: 0.3044 - val_acc: 0.8982
Epoch 3/20
6703/6703 [==============================] - 27s 4ms/step - loss: 0.2871 - acc: 0.8979 - val_loss: 0.3079 - val_acc: 0.8982
Epoch 4/20
6703/6703 [==============================] - 27s 4ms/step - loss: 0.2742 - acc: 0.8986 - val_loss: 0.3160 - val_acc: 0.8980
Epoch 5/20
6703/6703 [==============================] - 28s 4ms/step - loss: 0.2597 - acc: 0.8988 - val_loss: 0.3303 - val_acc: 0.8962
Epoch 6/20
6703/6703 [==============================] - 28s 4ms/step - loss: 0.2471 - acc: 0.9011 - val_loss: 0.3536 - val_acc: 0.8963
Epoch 7/20
6703/6703 [==============================] - 28s 4ms/step - loss: 0.2347 - acc: 0.9042 - val_loss: 0.3542 - val_acc: 0.8839
Epoch 8/20
6703/6703 [==============================] -

In [28]:
loss, accuracy = model1.evaluate(X_train,y_train)
print('Training Accuracy is {} '.format(accuracy*100))

6703/6703 [==============================] - 12s 2ms/step - loss: 0.1411 - acc: 0.9386
Training Accuracy is 93.85859966278076 


In [29]:
loss, accuracy = model1.evaluate(X_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

2235/2235 [==============================] - 4s 2ms/step - loss: 0.5883 - acc: 0.8655
Testing Accuracy is 86.54806613922119 


In [30]:
y_pred_train = model1.predict(X_train)

In [31]:
rounded_train = [round(x[0]) for x in y_pred_train]

In [34]:
confusion_matrix(y_train, rounded_train)

array([[ 10272,  11558],
       [  1615, 191050]])

In [35]:
y_pred = model1.predict(X_test)

In [36]:
rounded = [round(x[0]) for x in y_pred]

In [37]:
confusion_matrix(y_test, rounded)

array([[  589,  6688],
       [ 2930, 61292]])

# Convolution Neural Network

In [38]:
X1 = original_df['review']
y1 = original_df['target']
from sklearn.model_selection import train_test_split
X_train1, X_test1 , y_train1, y_test1 = train_test_split(X1, y1 , test_size = 0.20)

In [39]:

vocab_size = 5000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train1)

In [40]:
X_train_sequences = tokenizer.texts_to_sequences(X_train1)
X_test_sequences = tokenizer.texts_to_sequences(X_test1)

In [41]:
max_length = 100
padding_type = "post"
trunction_type="post"
X_train_padded = pad_sequences(X_train_sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunction_type)
X_test_padded = pad_sequences(X_test_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunction_type)

# Sequential Model

In [42]:
# import urllib.request

In [ ]:
# !curl http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [ ]:
# url = "http://nlp.stanford.edu/data/glove.6B.zip \
# -O /tmp/glove.6B.zip"

In [ ]:
# !wget --no-check-certificate \
# http://nlp.stanford.edu/data/glove.6B.zip \
# -O /tmp/glove.6B.zip

In [49]:
with zipfile.ZipFile('./glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/glove')

In [50]:
embeddings_index = {}
f = open('/tmp/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [51]:
embeddings_index.get("paper")

array([-0.85034  ,  0.33358  , -0.65889  , -0.49871  ,  0.36585  ,
       -0.19245  ,  0.25658  , -0.053408 ,  0.31474  ,  0.2443   ,
        0.29337  , -0.44917  ,  0.15175  ,  0.39314  , -0.31786  ,
        0.060525 ,  0.81775  , -0.38847  ,  0.76761  , -1.1041   ,
       -0.1544   ,  0.31655  , -0.37238  , -0.11485  ,  0.51635  ,
       -0.39289  ,  0.16301  , -0.2532   , -0.50976  ,  0.15201  ,
        0.27808  ,  0.52522  , -0.38815  , -0.3472   , -0.61818  ,
        0.17022  ,  0.12251  , -0.24191  , -0.38877  , -0.53176  ,
       -0.46987  , -0.70502  , -0.62126  , -0.38689  , -0.85637  ,
       -0.41003  , -0.47487  , -0.21083  , -0.81338  , -0.52398  ,
        0.49894  ,  0.37909  ,  0.55428  ,  1.123    , -0.42121  ,
       -1.5674   , -0.56892  ,  0.40819  ,  1.7949   ,  0.16856  ,
       -0.0029332,  0.28786  , -0.90088  , -0.094214 ,  0.79993  ,
       -0.39096  ,  0.76286  ,  0.71307  ,  0.13194  , -0.40756  ,
       -0.18687  ,  0.89562  ,  0.46867  , -0.0028801,  0.0253

In [52]:
embedding_matrix = np.zeros((len(X) + 1, max_length))
for word, i in X1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [53]:
embedding_layer = Embedding(input_dim=len(X) + 1,
                            output_dim=max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)


In [54]:
model = Sequential([
    embedding_layer,
  Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
  Dense(10, activation='relu'),
  Dense(1, activation='sigmoid'),
])

In [55]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [56]:
history = model.fit(X_train_padded, y_train1, epochs=20, validation_data=(X_test_padded, y_test1))

Epoch 1/20
7150/7150 [==============================] - 279s 39ms/step - loss: 0.4462 - accuracy: 0.8962 - val_loss: 0.3263 - val_accuracy: 0.8995
Epoch 2/20
7150/7150 [==============================] - 312s 44ms/step - loss: 0.3301 - accuracy: 0.8977 - val_loss: 0.3263 - val_accuracy: 0.8995
Epoch 3/20
7150/7150 [==============================] - 314s 44ms/step - loss: 0.3320 - accuracy: 0.8969 - val_loss: 0.3262 - val_accuracy: 0.8995
Epoch 4/20
7150/7150 [==============================] - 261s 36ms/step - loss: 0.3280 - accuracy: 0.8987 - val_loss: 0.3263 - val_accuracy: 0.8995
Epoch 5/20
7150/7150 [==============================] - 239s 33ms/step - loss: 0.3295 - accuracy: 0.8980 - val_loss: 0.3262 - val_accuracy: 0.8995
Epoch 6/20
7150/7150 [==============================] - 244s 34ms/step - loss: 0.3300 - accuracy: 0.8977 - val_loss: 0.3262 - val_accuracy: 0.8995
Epoch 7/20
7150/7150 [==============================] - 234s 33ms/step - loss: 0.3291 - accuracy: 0.8981 - val_loss: 0

In [57]:
loss, accuracy = model.evaluate(X_test_padded,y_test1)
print('Testing Accuracy is {} '.format(accuracy*100))

1788/1788 [==============================] - 16s 9ms/step - loss: 0.3262 - accuracy: 0.8995
Testing Accuracy is 89.9473786354065 
